In [130]:
import json
import folium
import numpy as np
import pandas as pd
import googlemaps
import matplotlib.pyplot as plt
import os
from folium.plugins import MarkerCluster

from bs4 import BeautifulSoup
import requests
import time

import warnings
warnings.filterwarnings("ignore")

In [51]:
df_seoul = pd.read_csv("C:\\Users\\MINHO\\education\\MC python\\projectdata\\df_seoul(euc-kr).csv", encoding="euc-kr")
df_info25 = pd.read_csv("C:\\Users\\MINHO\\education\\MC python\\projectdata\\df_rest25_dense(euc-kr).csv", encoding="euc-kr")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time


chromedriver = '/Users/datakim/workspace/selenium_learning/chromedriver' 
driver = webdriver.Chrome(executable_path="chromedriver.exe")


df_seoul['kakao_keyword'] = df_seoul['구'] + "%20" + df_seoul['업소명']
df_seoul['kakao_map_url'] = ''


for i, keyword in enumerate(df_seoul['kakao_keyword'].tolist()):
    print(i, f"/ {df_seoul.shape[0]} 행", keyword)
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        time.sleep(3.5)
        df_seoul.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):
            try:
                df_seoul.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df_seoul.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass    

In [ ]:
from tqdm.notebook import tqdm_notebook

blog_review_list = []

for i, url in enumerate(tqdm_notebook(df_seoul['kakao_map_url'])):
    if i < 1040:
        continue
    driver = webdriver.Chrome(executable_path="chromedriver.exe")
    print(i, url)
        
    driver.get(url)
    time.sleep(1)
    review_text = ""
    
    try:
        
        kakao_map_name = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2").text
        blog_review_qty = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(5) > span").text
    except Exception as e1:
        if "inactive" in str(e1):
            print("크롤링 문제 발생")
            
    num = 0
    try:
        while num < 3:
            try:
                num += 1
                review_text = review_text + driver.find_element_by_css_selector(f"div.wrap_list > ul > li:nth-child({num}) > a > div.review_story > p").text
                
                if num == 3:
                    blog_review_list.append(review_text)
                    kakao_map_name_list.append(kakao_map_name)
                    break
            except Exception as e1:
                if "li:nth-child(1)" in str(e1):
                    print(f"{i}행 리뷰없음")
                    review_text = "empty"
                    blog_review_list.append(review_text)
                    kakao_map_name_list.append(kakao_map_name)
                    break
                else:
                    print(f"{i}행 문제 발생 - 총 리뷰 {num - 1}개")
                    blog_review_list.append(review_text)     
                    kakao_map_name_list.append(kakao_map_name)
                    break
                
    except Exception as e2:
        print(e2)
        print(f"{i}행 문제 발생")
    driver.quit()


df_seoul['리뷰내용'] = blog_review_list